## 005 Reinsforment Leearning - Deep Q Learning Trading for Microsoft (MSFT)

#### Autores: Adrián Herrera, Patrick F. Bárcena y Carlos Moreno


### Marco Teórico:

El **`Aprendizaje por Refuerzo (Reinforcement Learning, RL)`** es un paradigma de aprendizaje automático en el que un agente aprende a tomar decisiones secuenciales a través de la interacción con un entorno dinámico (ambiente). En lugar de utilizar datos etiquetados como en el aprendizaje supervisado, el agente explora distintas acciones, recibe recompensas o penalizaciones, y mejora progresivamente su estrategia para maximizar el retorno acumulado. 

Statquest hace un gran trabajo explicando esto, poniendo como ejemplo tomar la decisión probabilística de ir ya sea a Burguer King o M'cdonals por papas fritas, y que el desenlace positivo o negativo (reward) a la asisitencia aleatoria, será la variable que determine que tan probable será seguir asistiendo al lugar o no. 

En el contexto de **`trading algorítmico`**, RL ofrece una forma poderosa de entrenar agentes que aprenden cuándo **`comprar, vender o mantener`** activos financieros para optimizar métricas como beneficios acumulados, Sharpe ratio o drawdowns.

---

### ✅ **`¿Por qué usar RL/DQL en trading?`**
- 🔄 **Secuencialidad:** Permite al agente aprender decisiones encadenadas (*¿vendo ahora o espero un día más?*).  
- 📈 **Adaptabilidad:** Puede ajustarse a cambios dinámicos del mercado.  
- 🎯 **Exploración vs explotación:** Balancea entre probar estrategias nuevas y optimizar las conocidas.  

---

### 🆚 **`Q-Learning vs Deep Q-Learning`**
|                     | Q-Learning                        | Deep Q-Learning (DQL)           |
|---------------------|-------------------------------------|-----------------------------------|
| 🔢 **Representación** | Tabla Q discreta                  | Red neuronal para estimar valores Q |
| 🧠 **Escalabilidad**   | Limitada (no funciona bien con muchos estados) | Escalable a espacios de estados grandes |
| 🕒 **Entrenamiento**   | Rápido                            | Más pesado (requiere más cómputo) |
| 📈 **Aplicación**      | Ambientes simples                 | Ambientes complejos (como trading real) |

---

### 💡 **`Contexto del proyecto`**
En este proyecto desarrollamos un agente de trading utilizando **Deep Q-Learning (DQL)** para aprender a operar sobre datos históricos de **META (MSFT)**. Nuestro objetivo es evaluar cómo un agente entrenado mediante RL se compara con una estrategia pasiva como *Buy & Hold* y analizar sus ventajas y limitaciones.



### 📖 Librerías

In [18]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim

### 👨🏻‍💻 Importación de Datos:

In [10]:
# Descargamos 5 años de datos diarios
df = yf.download("MSFT", start="2018-01-01", end="2023-12-31")

# Guardamos en CSV
df.to_csv("data/MSFT_5yr.csv")

print("✅ Datos de MSFT guardados en data/MSFT_5yr.csv")


[*********************100%***********************]  1 of 1 completed

✅ Datos de MSFT guardados en data/MSFT_5yr.csv


### ⚒️ Definición de Estados y Acciones:

Acciones posibles (Action Space):

0 = Hold (mantener)

1 = Buy (comprar)

2 = Sell (vender)

In [11]:
# Normalizamos precios de cierre entre 0 y 1

scaler = MinMaxScaler()
df["Close_Normalized"] = scaler.fit_transform(df[["Close"]])

# Añadimos columna de posición actual (inicialmente 0 = sin posición)
df["Position"] = 0

# Definimos espacio de acciones
actions = {0: "Hold", 1: "Buy", 2: "Sell"}

print("✅ Estados y acciones definidos.")
df[["Close", "Close_Normalized", "Position"]].head()


✅ Estados y acciones definidos.


Price,Close,Close_Normalized,Position
Ticker,MSFT,,
Date,,,
2018-01-02,79.328529,0.002892,0
2018-01-03,79.697716,0.004123,0
2018-01-04,80.399185,0.006462,0
2018-01-05,81.395943,0.009785,0
2018-01-08,81.479034,0.010062,0


### 🧠 Clase TradingEnvironment

Creamos el simulador previo a la partición de los datos para el modelo tenga bases para jugar y experimentar.

In [ ]:
class TradingEnvironment:
    def __init__(self, data, initial_balance=10000):
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no posición, 1 = comprado
        self.shares_held = 0
        self.total_asset = self.balance
        self.done = False
        return self._get_state()

    def _get_state(self):
        price = float(self.data.loc[self.current_step, "Close_Normalized"].iloc[0])
        return np.array([price, self.position])

    def step(self, action):
        price = self.data.loc[self.current_step, "Close"]
        reward = 0

        if action == 1 and self.position == 0:  # Buy
            self.shares_held = self.balance // price
            self.balance -= self.shares_held * price
            self.position = 1
        elif action == 2 and self.position == 1:  # Sell
            self.balance += self.shares_held * price
            self.shares_held = 0
            self.position = 0
            reward = self.balance - self.initial_balance
        else:
            # Hold or invalid action
            reward = 0

        self.total_asset = self.balance + self.shares_held * price
        self.current_step += 1

        if self.current_step >= len(self.data) - 1:
            self.done = True

        return self._get_state(), reward, self.done


### 🏋🏼‍♂️  Entrenamiento del entorno (Q Learning Clásico - prueba)

Hacemos una prueba rápida del entorno con un algoritmo de Q-Learning clásico para ver si funciona correctamente.

In [16]:
# 📚 Tabla Q inicial
q_table = {}

# Hiperparámetros
alpha = 0.1
gamma = 0.95
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01
episodes = 100

env = TradingEnvironment(df)

for ep in range(episodes):
    state = tuple(env.reset())
    total_reward = 0

    while not env.done:
        # Epsilon-greedy: Explorar o explotar
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice([0, 1, 2])  # Explorar
        else:
            action = q_table.get(state, np.zeros(3)).argmax()  # Explotar

        next_state, reward, done = env.step(action)
        next_state = tuple(next_state)

        # Actualizar tabla Q
        old_q = q_table.get(state, np.zeros(3))[action]
        next_max = q_table.get(next_state, np.zeros(3)).max()

        new_q = (1 - alpha) * old_q + alpha * (reward + gamma * next_max)

        q_table.setdefault(state, np.zeros(3))
        q_table[state][action] = new_q

        state = next_state
        total_reward += reward

    # Decay de epsilon
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay

    print(f"🎯 Episodio {ep+1}/{episodes} | Recompensa Total: {float(total_reward):.2f}")

print("✅ Entrenamiento Q-Learning básico completado.")


/var/folders/zq/qf312yqs5wnb29fvj3txfrqm0000gn/T/ipykernel_11018/1693972736.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(self.data.loc[self.current_step, "Close_Normalized"])
/var/folders/zq/qf312yqs5wnb29fvj3txfrqm0000gn/T/ipykernel_11018/3837540229.py:35: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  q_table[state][action] = new_q
/var/folders/zq/qf312yqs5wnb29fvj3txfrqm0000gn/T/ipykernel_11018/3837540229.py:44: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"🎯 Episodio {ep+1}/{episodes} | Recompensa Total: {float(total_reward):.2f}")


🎯 Episodio 1/100 | Recompensa Total: 1693134.77
🎯 Episodio 2/100 | Recompensa Total: 1747670.17
🎯 Episodio 3/100 | Recompensa Total: 1396757.35
🎯 Episodio 4/100 | Recompensa Total: 2062276.27
🎯 Episodio 5/100 | Recompensa Total: 3130830.21
🎯 Episodio 6/100 | Recompensa Total: 3341704.05
🎯 Episodio 7/100 | Recompensa Total: 1807261.99
🎯 Episodio 8/100 | Recompensa Total: 545052.57
🎯 Episodio 9/100 | Recompensa Total: 4566.41
🎯 Episodio 10/100 | Recompensa Total: 3346237.19
🎯 Episodio 11/100 | Recompensa Total: 2064567.73
🎯 Episodio 12/100 | Recompensa Total: 4271816.61
🎯 Episodio 13/100 | Recompensa Total: 2715026.75
🎯 Episodio 14/100 | Recompensa Total: 1893919.75
🎯 Episodio 15/100 | Recompensa Total: 3507437.13
🎯 Episodio 16/100 | Recompensa Total: 1421627.44
🎯 Episodio 17/100 | Recompensa Total: 196840.67
🎯 Episodio 18/100 | Recompensa Total: 2290460.81
🎯 Episodio 19/100 | Recompensa Total: 1092366.70
🎯 Episodio 20/100 | Recompensa Total: 1812485.10
🎯 Episodio 21/100 | Recompensa Tot

## 📊 **Análisis de resultados: Q-Learning clásico**

Durante los 100 episodios de entrenamiento con el agente Q-Learning clásico, observamos una evolución interesante en las recompensas acumuladas:  

  - Las recompensas muestran una alta variabilidad y algunos valores extremadamente bajos. Esto se debe a que el agente está en la fase de **exploración (ε-greedy)**, probando acciones al azar para aprender sobre el entorno.  

- **Mejora progresiva (Episodios 21-80):**
  - También, mpiezan a aparecer episodios con recompensas significativamente más altas (>3M), lo cual indica que el agente comienza a identificar **estrategias básicas rentables**.  

- 
  - Hacia los últimos episodios, se observa un aumento notable en las recompensas acumuladas (algunos episodios superan los 8M). Esto sugiere que el agente está **explotando mejor las políticas aprendidas** para maximizar beneficios. Igual son solo 100 episodios, seguimos creciendo como agentes.


###  **Próximos pasos**
En la siguiente fase desarrollaremos un **agente DQL con red neuronal**, lo que permitirá:  
- Aproximar funciones Q para espacios de estados continuos.  
- Mejorar la capacidad de generalización del agente.  
- Evaluar su rendimiento frente al Q-Learning clásico y una estrategia *Buy & Hold*.  



In [23]:
# 📥 Importar el agente DQL desde utils
from utils.rl_agent import DQLAgent

# 📦 Parámetros
state_size = 2  # [precio_normalizado, posición]
action_size = 3  # Buy, Sell, Hold
agent = DQLAgent(state_size, action_size)
episodes = 100
batch_size = 32

env = TradingEnvironment(df)

# 🚀 Entrenamiento
for ep in range(episodes):
    state = env.reset()
    total_reward = 0

    while not env.done:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

    print(f"🎯 Episodio {ep+1}/{episodes} | Recompensa Total: {total_reward:.2f}")

print("✅ Entrenamiento DQL (versión rápida) completado.")


/var/folders/zq/qf312yqs5wnb29fvj3txfrqm0000gn/T/ipykernel_11018/1693972736.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(self.data.loc[self.current_step, "Close_Normalized"])


TypeError: can't assign a Series to a torch.FloatTensor